> Set up code to invoke RESTful APIs, produce and consume from Kafka

In [ ]:
import json
import os
from dataclasses import dataclass
from typing import Dict, Callable, Any
from kafka import KafkaConsumer, KafkaProducer
import requests
import logging

CONTENT_TYPE = 'application/json'

logging.basicConfig(level=logging.DEBUG)
KAFKA_ENDPOINT = 'bootstrap.kafka.partner-eph-6.tmachine.io:443'
CORE_API_URL = "https://core-api.partner-eph-6.tmachine.io"
WORKFLOW_API_URL = "https://workflows-api.partner-eph-6.tmachine.io"
X_AUTH_TOKEN = "A0006786022557907328897!rI1MCYa35TdEFt3kka7xh5edAoXEHfXGzntcA4vSxAseR+Cu+rseyz+j9Ql4WffZD8IsAZ9DUKDttPlqvSNsrfZd6To="

PIB_URL_BATCH_GET = "/v1/posting-instruction-batches:batchGet"
PIB_TOPIC = 'vault.api.v1.postings.posting_instruction_batch.created'
PIB_REQUEST_TOPIC = 'vault.core.postings.requests.v1'
GET_PAC = "/v1/postings-api-clients"

class TMApiClient:
    @dataclass
    class TMConnectionDetails:
        core_api_url: str
        workflow_api_url: str
        token: str
        kafka_url: str
        kafka_security_protocol: str = "SSL"
        content_type: str = CONTENT_TYPE

    def __init__(self, conn: TMConnectionDetails):
        self.connection_details = conn
        self.default_headers = {
            'Content-Type': conn.content_type,
            'X-Auth-Token': conn.token
        }

    def call_core_api(self, endpoint: str, query_params: Dict[str, str], method: str = 'get', body: Any = None,
                      headers: Dict[str, str] = {}):
        return self.__call_api(url=self.connection_details.core_api_url + endpoint,
                               query_params=query_params,
                               method=method,
                               body=body,
                               extra_headers=headers)

    def call_workflow_api(self, endpoint: str, query_params: Dict[str, str], method: str = 'get', body: Any = None,
                          headers: Dict[str, str] = {}):
        return self.__call_api(url=self.connection_details.workflow_api_url + endpoint,
                               query_params=query_params,
                               method=method,
                               body=body,
                               extra_headers=headers)

    def __call_api(self, url: str, query_params: Dict[str, str], method: str, body: Any, extra_headers: Dict[str, str]):
        response = requests.request(method=method,
                                    url=url,
                                    params=query_params,
                                    data=body,
                                    headers={**self.default_headers, **extra_headers}
                                    )
        if not response.ok:
            raise Exception('Failed getting response for request %s, %s %s' % (
                str(response.request), response.status_code, response.text))
        return response.json()

    def publish_kafka_message(self, topic: str, key: str, value: Any) -> None:
        """
        Publish message to particular kafka topic.
        :param topic: Topic to where send the message
        :param key: Key of the message converted to bytes
        :param value: Value of the message, that can be converted to json and the to bytes
        :return: None
        """
        producer = KafkaProducer(bootstrap_servers=[self.connection_details.kafka_url],
                                 security_protocol=self.connection_details.kafka_security_protocol,
                                api_version=(0, 8, 2))

        producer.send(topic, key=key.encode('utf-8'), value=json.dumps(value).encode('utf-8'))

    def subscribe_to_kafka_topic(self,
                                 topic: str,
                                 group_id: str = str(os.getpid()),
                                 offset: str = 'earliest',
                                 callback: Callable[[str, str], None] = lambda *args: None,
                                 ) -> None:
        """
        Subscribe particular kafka topic to receive messages in json format encoded in utf-8
        :param topic: topic name to subscribe the consumer
        :param group_id: configure group id of consumers. Default value is process Id.
        :param offset: configure the offset for consumer between earliest,latest or exact offset value.
        Default value is earliest.
        :param callback: function that consumes message as arguments of key and value.
        :return: None
        """
        consumer = KafkaConsumer(topic,
                                 group_id=group_id, auto_offset_reset=offset,
                                 bootstrap_servers=[self.connection_details.kafka_url],
                                 security_protocol=self.connection_details.kafka_security_protocol)
        logging.debug("Consumer for topic {} has started".format(topic))
        for record in consumer:
            logging.debug("%s:%d:%d: key=%s value=%s" % (record.topic, record.partition,
                                                         record.offset, record.key,
                                                         record.value))
            callback(record.key.decode('utf8'), json.loads(record.value.decode('utf8').replace("'", '"')))

details = TMApiClient.TMConnectionDetails(core_api_url=CORE_API_URL,
                                             workflow_api_url=WORKFLOW_API_URL,
                                             token=X_AUTH_TOKEN,
                                             kafka_url=KAFKA_ENDPOINT)

: 

> Call Core API to fetch a PIB

In [2]:
client = TMApiClient(details)
client.call_core_api(PIB_URL_BATCH_GET, {"ids": "35ca2636-bc89-4786-abd2-492becd7e004"})

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): core-api.partner-eph-6.tmachine.io
DEBUG:urllib3.connectionpool:https://core-api.partner-eph-6.tmachine.io:443 "GET /v1/posting-instruction-batches:batchGet?ids=35ca2636-bc89-4786-abd2-492becd7e004 HTTP/1.1" 200 932


{'posting_instruction_batches': {'35ca2636-bc89-4786-abd2-492becd7e004': {'id': '35ca2636-bc89-4786-abd2-492becd7e004',
   'create_request_id': '8072b747-77e3-4c86-a92b-0e5818f3c84f',
   'client_id': 'AsyncCreatePostingInstructionBatch',
   'client_batch_id': 'be92bf20-80f5-4be2-bda0-2dfea16bc569',
   'posting_instructions': [{'id': '0c8e26e0-23ac-4879-aa11-f641bc336966',
     'client_transaction_id': '5828OG7D',
     'inbound_hard_settlement': {'amount': '400',
      'denomination': 'EUR',
      'target_account': {'account_id': '40d78ae3-24f8-5393-251d-6ac5811c0433'},
      'internal_account_id': 'migration_sample',
      'advice': False,
      'target_account_id': '40d78ae3-24f8-5393-251d-6ac5811c0433'},
     'pics': [],
     'instruction_details': {'Contable_date': '14-01-2020',
      'Movement_date': '15-01-2020',
      'scheme_name': 'domestic_payment_scheme',
      'value_date': '12-01-2020'},
     'committed_postings': [{'credit': False,
       'amount': '400',
       'denominat

> Receive Posting Instruction Batches events, printing these where key is equal to "test"

In [29]:
client.subscribe_to_kafka_topic(topic=PIB_TOPIC,
                                    callback=lambda key, value: print("key %s value %s" % (key, value)) if (key == "test") else None)

DEBUG:kafka.metrics.metrics:Added sensor with name connections-closed
DEBUG:kafka.metrics.metrics:Added sensor with name connections-created
DEBUG:kafka.metrics.metrics:Added sensor with name select-time
DEBUG:kafka.metrics.metrics:Added sensor with name io-time
DEBUG:kafka.client:Initiating connection to node bootstrap-0 at bootstrap.kafka.partner-eph-6.tmachine.io:443
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-sent-received
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-sent
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-received
DEBUG:kafka.metrics.metrics:Added sensor with name request-latency
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.bytes-sent
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.bytes-received
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.latency
DEBUG:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=bootstrap.kafka.partner-eph-6.tmachine.io:443 <disconnected> [u

NoBrokersAvailable: NoBrokersAvailable

> Publish request for to Posting API

In [3]:
client = TMApiClient(details)
request = json.loads("""{
"request_id":"test",
"posting_instruction_batch":{
  "client_batch_id":"test",
  "posting_instructions":[
     {
        "client_transaction_id":"f9ea38b6-59a8-497d-9a43-dd907adb28d6",
        "instruction_details":null,
        "override":null,
        "transaction_code":null,
        "outbound_authorisation":null,
        "inbound_authorisation":null,
        "authorisation_adjustment":null,
        "settlement":null,
        "release":null,
        "inbound_hard_settlement":{
            "amount":"21",
            "denomination":"USD",
            "target_account":{
                "account_id":"40d78ae3-24f8-5393-251d-6ac5811c0433"
            },
            "internal_account_id":"migration_sample",
            "advice":true

        }
     }
  ],
  "batch_details":{
     "force_override":"true"
  }
}}
""")

client.publish_kafka_message(PIB_REQUEST_TOPIC, "40d78ae3-24f8-5393-251d-6ac5811c0433", request)

DEBUG:kafka.producer.kafka:Starting the Kafka producer
DEBUG:kafka.metrics.metrics:Added sensor with name connections-closed
DEBUG:kafka.metrics.metrics:Added sensor with name connections-created
DEBUG:kafka.metrics.metrics:Added sensor with name select-time
DEBUG:kafka.metrics.metrics:Added sensor with name io-time
DEBUG:kafka.client:Initiating connection to node bootstrap-0 at bootstrap.kafka.partner-eph-6.tmachine.io:443
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-sent-received
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-sent
DEBUG:kafka.metrics.metrics:Added sensor with name bytes-received
DEBUG:kafka.metrics.metrics:Added sensor with name request-latency
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.bytes-sent
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.bytes-received
DEBUG:kafka.metrics.metrics:Added sensor with name node-bootstrap-0.latency
DEBUG:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=bootstra

NoBrokersAvailable: NoBrokersAvailable

> get list of posting api client

In [35]:
import requests

url = "https://core-api.partner-eph-6.tmachine.io/v1/postings-api-clients"

querystring = {"page_size":"100"}

payload = ""
headers = {
    'Content-Type': "application/json",
    'X-Auth-Token': "A0006786022557907328897!rI1MCYa35TdEFt3kka7xh5edAoXEHfXGzntcA4vSxAseR+Cu+rseyz+j9Ql4WffZD8IsAZ9DUKDttPlqvSNsrfZd6To=",
    'cache-control': "no-cache",
    'Postman-Token': "36c77889-37a2-43be-8466-14969a3d73fb"
    }

response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

print(json.dumps(json.loads(response.text), indent = 1))

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): core-api.partner-eph-6.tmachine.io:443
DEBUG:urllib3.connectionpool:https://core-api.partner-eph-6.tmachine.io:443 "GET /v1/postings-api-clients?page_size=100 HTTP/1.1" 200 588


{
 "postings_api_clients": [
  {
   "id": "AsyncCreatePostingInstructionBatch",
   "response_topic": "vault.core.postings.async_creation_api.responses",
   "response_topic_low_priority": ""
  },
  {
   "id": "Migration",
   "response_topic": "vault.migrations.postings.responses",
   "response_topic_low_priority": ""
  },
  {
   "id": "PostingsApiTestQueue3_test_4",
   "response_topic": "integration.testone.core.postings.test.test.responses",
   "response_topic_low_priority": ""
  },
  {
   "id": "DlscPostingClientID",
   "response_topic": "integration.postings_api.dlsc.response",
   "response_topic_low_priority": ""
  }
 ],
 "previous_page_token": "",
 "next_page_token": ""
}
